# EXPLORATORY DATA ANALYSIS ON NYC TAXI TRIP DURATION #


## Importing libraries ##


In [145]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance
import warnings
from datetime import timedelta
import datetime as dt
warnings.filterwarnings('ignore')

## Import data ##


In [146]:
data = pd.read_csv('nyc_taxi_trip_duration.csv')


## STRUCTURE OF DATA AND CONTENT ##

In [147]:
data.shape


(729322, 11)

### we can see that there are 729322 rows and 11 columns ###



## Data collection ##


The variables present are listed as follows :


 - **id** - a unique identifier for each trip
 - **vendor_id** - a code indicating the provider associated with the trip record
 - **pickup_datetime** - date and time when the meter was engaged
 - **dropoff_datetime** - date and time when the meter was disengaged
 - **passenger_count** - the number of passengers in the vehicle (driver entered value)
 - **pickup_longitude** - the longitude where the meter was engaged
 - **pickup_latitude** - the latitude where the meter was engaged
 - **dropoff_longitude** - the longitude where the meter was disengaged
 - **dropoff_latitude** - the latitude where the meter was disengaged
 - **store_and_fwd_flag** - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip)
 - **trip_duration** - (target) duration of the trip in seconds

 
 
 It is to be noted that the dropoff_datetime and store_and_fwd_flag are not present at the start of the journey 

 ### Checking the columns for what all variables are present and it can be confirmed that we have all the variables  as mentioned in the project ###

In [148]:
data.columns

Index(['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag',
       'trip_duration'],
      dtype='object')

### Looking at the data we can see what all datatypes that are present and get a general overview ###

In [149]:
# a view of datatypes that are present
data.dtypes

id                     object
vendor_id               int64
pickup_datetime        object
dropoff_datetime       object
passenger_count         int64
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dtype: object

In [150]:
# sample of first 5 rows to understand how values are represented
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [151]:
# sample of last 5 rows to recognise any trends
data.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
729317,id3905982,2,2016-05-21 13:29:38,2016-05-21 13:34:34,2,-73.965919,40.789780,-73.952637,40.789181,N,296
729318,id0102861,1,2016-02-22 00:43:11,2016-02-22 00:48:26,1,-73.996666,40.737434,-74.001320,40.731911,N,315
729319,id0439699,1,2016-04-15 18:56:48,2016-04-15 19:08:01,1,-73.997849,40.761696,-74.001488,40.741207,N,673
729320,id2078912,1,2016-06-19 09:50:47,2016-06-19 09:58:14,1,-74.006706,40.708244,-74.013550,40.713814,N,447
729321,id1053441,2,2016-01-01 17:24:16,2016-01-01 17:44:40,4,-74.003342,40.743839,-73.945847,40.712841,N,1224


## REFORMATTING FEATURES AND CHECKING CONSISTENCY ##

## Checking for missing values ##

In [152]:
data.isnull().sum()

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

**There is no null values present hence we dont have to assign any values**


### Analysing each variable datatypes

In [153]:
# INTEGER DATATYPE

data.dtypes[data.dtypes == 'int64']

vendor_id          int64
passenger_count    int64
trip_duration      int64
dtype: object

**observations:**

**vendor_id** are the provider identification thus it is considered categorical

**passenger_count** can be take as categorical for better analysis

**trip_duration** are numbers hence they are also fall in integer category

In [154]:
# typecasting passenger count to category
data['passenger_count'] = data['passenger_count'].astype('category')


In [155]:
# FLOAT DATATYPE
data.dtypes[data.dtypes == 'float64']

pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
dtype: object

**observations:**

**pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude** are suited to in float datatype

In [156]:
# OBJECT DATATYPE
data.dtypes[data.dtypes == 'object']

id                    object
pickup_datetime       object
dropoff_datetime      object
store_and_fwd_flag    object
dtype: object

**Observations:**

**id** is a unique variable with numerical parts and id written at front and I choose to treat it as an object itself

**pickup_datetime and dropoff_datetime** are datetime variable

**store_and_fwd_flag** can be grouped under categorical

In [157]:
# typecasting store_and_fwd_flag to category
data['store_and_fwd_flag'] = data['store_and_fwd_flag'].astype('category')

In [158]:
# Convert into datetime DATA TYPE from strings
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [159]:
#Using datetime to find and create new columns - day_name , month, day_num, pickup_hour

data['day_name'] = data.pickup_datetime.dt.day_name()
data['month'] = data.pickup_datetime.dt.month
data['day_num'] = data.pickup_datetime.dt.weekday
data['pickup_hour'] = data.pickup_datetime.dt.hour

In [160]:
#typecasting month, day_num, pickup_hour to category
data['month'] = data['month'].astype('category')
data['day_num'] = data['day_num'].astype('category')
data['pickup_hour'] = data['pickup_hour'].astype('category')

In [161]:
#function to find the distance traversed
def calc_distance(df):
    pickup = (df['pickup_latitude'], df['pickup_longitude'])
    drop = (df['dropoff_latitude'], df['dropoff_longitude'])
    return geopy.distance.distance(pickup, drop).km

In [162]:
#adding distance column in the dataframe
data['distance'] = data.apply(lambda x: calc_distance(x), axis = 1)

In [163]:
#calculating the data and imputing the speed into the dataframe
data['speed'] = (data.distance/(data.trip_duration/3600))

In [164]:
#the variable datatypes are analysed
data.dtypes

id                            object
vendor_id                      int64
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count             category
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag          category
trip_duration                  int64
day_name                      object
month                       category
day_num                     category
pickup_hour                 category
distance                      object
speed                         object
dtype: object

In [165]:
#all the categorical variable are converted to numerical

dummy = pd.get_dummies(data.store_and_fwd_flag, prefix='flag')
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.vendor_id, prefix='vendor_id')
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.month, prefix='month')
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.day_num, prefix='day_num')
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.pickup_hour, prefix='pickup_hour')
data = pd.concat([data,dummy], axis = 1)

dummy = pd.get_dummies(data.passenger_count, prefix='passenger_count')
data = pd.concat([data,dummy], axis = 1)


In [166]:
pd.set_option('display.max_columns', None)


In [167]:
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,day_name,month,day_num,pickup_hour,distance,speed,flag_N,flag_Y,vendor_id_1,vendor_id_2,month_1,month_2,month_3,month_4,month_5,month_6,day_num_0,day_num_1,day_num_2,day_num_3,day_num_4,day_num_5,day_num_6,pickup_hour_0,pickup_hour_1,pickup_hour_2,pickup_hour_3,pickup_hour_4,pickup_hour_5,pickup_hour_6,pickup_hour_7,pickup_hour_8,pickup_hour_9,pickup_hour_10,pickup_hour_11,pickup_hour_12,pickup_hour_13,pickup_hour_14,pickup_hour_15,pickup_hour_16,pickup_hour_17,pickup_hour_18,pickup_hour_19,pickup_hour_20,pickup_hour_21,pickup_hour_22,pickup_hour_23,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7,passenger_count_9
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400,Monday,2,0,16,1.1997699136961533 km,10.79792922326538 km,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100,Friday,3,4,23,4.123945167669728 km,13.496547821464564 km,1,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635,Sunday,2,6,17,7.250435807469041 km,15.964262328372202 km,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141,Tuesday,1,1,9,2.3582865466279324 km,7.440693749220471 km,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848,Wednesday,2,2,6,4.328155268197869 km,18.37424406310416 km,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
